### Task 1: Static Network Analysis
* Degree (in-degree/out-degree)
* Diameter
* <i>(Mean shortest path length)
* (Top-k eigenvalues)
* (Betweennes centrality)
* (Closeness centrality)</i>

A temporal network is goint to be analysed which consists of 678907 vertices and 4729035 edges, where each edge has time information associated with it. Some of the edges have the same source and target vertex, but are association with different timestamps. This statick network analysi, however, ignores time-information and thus we use a graph build solely on the pairs of source and target vertices.

### <font color="darkgreen">Imports, configuation and preprocessing</font>

In [124]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import math

#### Directed Graph with parallel edges (due to different timestamps of the edges between pairs of vertices)

In [2]:
wiki = open("data/tgraph_real_wikiedithyperlinks_noTime.txt", 'rb')
G_par = nx.read_edgelist(wiki, create_using=nx.MultiDiGraph())
wiki.close()

#### Directed Graph used for static network analysis

In [3]:
wiki = open("data/tgraph_real_wikiedithyperlinks_noTime.txt", 'rb')
G = nx.read_edgelist(wiki, create_using=nx.DiGraph())
wiki.close()

In [4]:
def create_dataframe(G_in):
    df_edge_in = pd.DataFrame(list(G_in.in_degree()), columns=['node', 'in edges'])
    df_edge_out = pd.DataFrame(list(G_in.out_degree()), columns=['node', 'out edges'])
    df_total = pd.merge(df_edge_in, df_edge_out, on='node')
    df_total.index = df_total.index + 1
    return df_total

In [5]:
wiki_df_par = create_dataframe(G_par)

In [86]:
wiki_df = create_dataframe(G)
wiki_df.head()

,node,in edges,out edges
1,1,28,88
2,6,10,0
3,8,62,6
4,9,1375,477
5,3,1068,451


### <font color="darkgreen">1. Static Network Analysis:</font> Degree distribution

The first network property which is going to be analyzed of the static network is the degree distribution. While it is an easily comprehensible network measure, it can still lead to interesting findings as we have seen in the previous assignment.

In [7]:
wiki_df['in edges'].mean() == wiki_df['out edges'].mean()

True

Which is expected of course since every outgoing edge is an incoming edge of the graph. This check is just done to check whether the parsing of the Graph object to a dataframe is done without any errors.

In [8]:
wiki_df['in edges'].mean()

5.3091660566174745

In [9]:
wiki_df['in edges'].max(), wiki_df['in edges'].min()

(10521, 0)

In [10]:
wiki_df['out edges'].max(), wiki_df['out edges'].min()

(4302, 0)

The mean of the incoming edges is just under above the 5 links. The node with the highest number of incoming edges has a little more than $10^4$ incoming edges, while the node with the highest number of outgoing edges has around  $4.3*10^3$ outgoing edges. These values do not implicate anything by themselves, but these values will be used in elaborations further on.

Before we continue with the static network analysis note that for the static network analysis it does not make sense to consider parallel edges. The parallel edges are edges between a pair of vertices at different time-stamps. But this gives a deceptive view in the static network analysis part. If a page has $y$ incoming edges of the same source vertex $x$, at different time-stamps it does not make sense to consider all of these parallel edges for many measures such as the degree distribution, page-rank etc. The choice is, therefore, made to continue with the network as a directed graph ($DiGraph$ in networkx) instead of a multi-directed graph ($MultiDiGraph$) for the static network analysis. These edges, however, have to be considered in part two for the Temporal Network Analysis where the time information of the edges is of the essence and techniques such as snapshot-based analysis can or even should be exploited.

Just to give you an overview of the deceiving effects it can have on the measures calculated over these two types of networks:

In [13]:
wiki_df_par['in edges'].mean(), wiki_df_par['in edges'].max(), wiki_df_par['out edges'].max()

(6.965659508592488, 15871, 15017)

It adds up the parallel edges between pair of vertices which result in a higher number of incoming and/or outgoing edges for many vertices. 

Let's continue with the analysis of the degree-distribution now that we have this pecularity out of the way. We are going to divide nodes of the network into deciles to get a better grasp of the characteristics of the top and bottom nodes when it boils down to number of incoming or outgoing edges.

In [63]:
wiki_df['decile_incoming_edges'] = pd.cut((wiki_df['in edges']), 10, labels=False)
wiki_df.loc[wiki_df.decile_incoming_edges.between(1, 9)].shape, wiki_df.loc[wiki_df.decile_incoming_edges.between(0, 1)].shape

((220, 4), (678830, 4))

In [133]:
wiki_df['decile_outgoing_edges'] = pd.cut((wiki_df['out edges']), 10, labels=False)
wiki_df.loc[wiki_df.decile_outgoing_edges.between(1, 9)].shape, wiki_df.loc[wiki_df.decile_outgoing_edges.between(0, 1)].shape

((305, 4), (678835, 4))

We see that there is a very skewed distribution among the nodes when we make splits based upon their number number of incoming or outgoing edges. There is thus a small set of nodes (i.e. pages) with relatively high number of incoming edges and a small set of nodes with a relatively high number of outgoing edges.

Let's divide the nodes in four groups such that in each quartile we have the same number of vertices (not the same as quartiles) intstead of the current configuration since this will faciliate a better comparison of the the groups considering the very skewed distribution. 

In [135]:
sorted_incoming_edges = wiki_df.sort_values(['in edges']).reset_index(drop = True)
sorted_outgoing_edges = wiki_df.sort_values(['out edges']).reset_index(drop = True)
sorted_incoming_edges.tail()

,node,in edges,out edges,decile_outgoing_edges
678902,300,7975,1686,3
678903,3546,8561,0,0
678904,146,9264,0,0
678905,149,9655,545,1
678906,394,10521,1777,4


In [156]:
incoming_edges_q1 = sorted_incoming_edges.iloc[: math.floor(sorted_incoming_edges.shape[0] / 4)]
incoming_edges_q2 = sorted_incoming_edges.iloc[math.floor(sorted_incoming_edges.shape[0] / 4) : 2 * (math.floor(sorted_incoming_edges.shape[0] / 4))]
incoming_edges_q3 = sorted_incoming_edges.iloc[2 * (math.floor(sorted_incoming_edges.shape[0] / 4)) : 3 * (math.floor(sorted_incoming_edges.shape[0] / 4))]
incoming_edges_q4 = sorted_incoming_edges.iloc[3 * (math.floor(sorted_incoming_edges.shape[0] / 4)) :]

incoming_edges_bottom_1 = sorted_incoming_edges.iloc[: math.floor(sorted_incoming_edges.shape[0] / 100)]
incoming_edges_top_1 = sorted_incoming_edges.iloc[99 * math.floor(sorted_incoming_edges.shape[0] / 100):]

In [160]:
outgoing_edges_q1 = sorted_outgoing_edges.iloc[: math.floor(sorted_outgoing_edges.shape[0] / 4)]
outgoing_edges_q2 = sorted_outgoing_edges.iloc[math.floor(sorted_outgoing_edges.shape[0] / 4) : 2 * (math.floor(sorted_outgoing_edges.shape[0] / 4))]
outgoing_edges_q3 = sorted_outgoing_edges.iloc[2 * (math.floor(sorted_outgoing_edges.shape[0] / 4)) : 3 * (math.floor(sorted_outgoing_edges.shape[0] / 4))]
outgoing_edges_q4 = sorted_outgoing_edges.iloc[3 * (math.floor(sorted_outgoing_edges.shape[0] / 4)) :]

outgoing_edges_bottom_1 = sorted_outgoing_edges.iloc[: math.floor(sorted_outgoing_edges.shape[0] / 100)]
outgoing_edges_top_1 = sorted_outgoing_edges.iloc[99 * math.floor(sorted_outgoing_edges.shape[0] / 100):]

And just to double-check

In [129]:
(incoming_edges_q4.shape[0] + incoming_edges_q3.shape[0] + 
 incoming_edges_q2.shape[0] + incoming_edges_q1.shape[0]) == sorted_incoming_edges.shape[0]

True

Let's have a look at what there are some notable things in the devised groups

In [145]:
incoming_edges_q1['in edges'].mean(), incoming_edges_q2['in edges'].mean(), incoming_edges_q3['in edges'].mean(), incoming_edges_q4['in edges'].mean()

(0.0, 0.806753237571144, 1.8176236993742856, 18.61205215372741)

In [154]:
incoming_edges_q1['out edges'].mean(), incoming_edges_q2['out edges'].mean(), incoming_edges_q3['out edges'].mean(), incoming_edges_q4['out edges'].mean()

(2.537313432835821, 2.36138835534921, 3.578950779491651, 11.466202004371675)

In [143]:
outgoing_edges_q1['in edges'].mean(), outgoing_edges_q2['in edges'].mean(), outgoing_edges_q3['in edges'].mean(), outgoing_edges_q4['in edges'].mean()

(3.9767507629944734, 3.667994296689959, 2.489977964483933, 11.101838813638212)

In [141]:
outgoing_edges_q1['out edges'].mean(), outgoing_edges_q2['out edges'].mean(), outgoing_edges_q3['out edges'].mean(), outgoing_edges_q4['out edges'].mean()

(0.0, 0.8123446024769334, 2.209997289749361, 18.214094232570744)

In [158]:
incoming_edges_bottom_1['in edges'].mean(), incoming_edges_top_1['in edges'].mean()

(0.0, 235.2441141848146)

In [159]:
incoming_edges_bottom_1['out edges'].mean(), incoming_edges_top_1['out edges'].mean()

(4.4613345117101195, 63.97998822836963)

In [161]:
outgoing_edges_bottom_1['in edges'].mean(), outgoing_edges_top_1['in edges'].mean()

(2.8858447488584473, 112.43952324896998)

In [162]:
outgoing_edges_bottom_1['out edges'].mean(), outgoing_edges_top_1['out edges'].mean()

(0.0, 160.2302825191289)

#### <i>Mean incoming and outgoing edges total network: $5.3$</i> 
#### <i>Sorted on number of incoming edges</i>
<table>
    <tr>
        <th>Group</th>
        <th>Mean #Incoming Edges</th>
        <th>Mean #Outgoing Edges</th>
    </tr>
    <tr>
        <td><i>Bottom 1%</i></td>
        <td>$0.00$</td>
        <td>$4.46$</td>
    </tr>
    <tr>
        <td>Q1 <i>(Bottom 25%)</i></td>
        <td>$0.00$</td>
        <td>$3.83$</td>
    </tr>
    <tr>
        <td>Q2</td>
        <td>$0.81$</td>
        <td>$2.36$</td>
    </tr>
    <tr>
        <td>Q3</td>
        <td>$1.82$</td>
        <td>$3.58$</td>
    </tr>
        <tr>
        <td>Q4 <i>(Top 25%)</i></td>
        <td>$18.6$</td>
        <td>$11.5$</td>
    </tr>
        <tr>
        <td><i>Top 1%</i></td>
        <td>$235$</td>
        <td>$64.0$</td>
    </tr>
</table>


#### <i>Sorted on number of outgoing edges</i>
<table>
    <tr>
        <th>Group</th>
        <th>Mean #Incoming Edges</th>
        <th>Mean #Outgoing Edges</th>
    </tr>
        <tr>
        <td><i>Bottom 1%</i></td>
        <td>$2.89$</td>
        <td>$0.00$</td>
    </tr>
    <tr>
        <td>Q1 <i>(Bottom 25%)</i></td>
        <td>$3.97$</td>
        <td>$0.00$</td>
    </tr>
    <tr>
        <td>Q2</td>
        <td>$3.67$</td>
        <td>$0.81$</td>
    </tr>
    <tr>
        <td>Q3</td>
        <td>$2.49$</td>
        <td>$2.21$</td>
    </tr>
        <tr>
        <td>Q4 <i>(Top 25%)</i></td>
        <td>$11.1$</td>
        <td>$18.2$</td>
    </tr>
        <tr>
        <td><i>Top 1%</i></td>
        <td>$112$</td>
        <td>$160$</td>
    </tr>
</table>


There are two noteworthy things in this graph. The nodes with a relatively high number of outgoing edges (i.e. the top $25\%$ or even the top $1\%$) also have a relatively high number of incoming edges when compared to the rest of the network. It thus seems that pages that have a lot of links to other pages also seem to be relatively high linked (i.e. have incoming edges) pages themselves  Note that there is a relatively large group (i.e. pages). The same can be seen in the top groups of when grouped/sorted by number of incoming edges. These groups do not only have a relatively higher number of incoming edges (on average) than the rest of the network, but also a relatively higher frequency of   in the network that do not have incoming edges, but have a relatively high number of outgoing edges.

In [169]:
print(str((wiki_df.loc[wiki_df["out edges"] == 0.00].shape[0] / wiki_df["out edges"].shape[0]) * 100), "%")
print(str((wiki_df.loc[wiki_df["in edges"] == 0.00].shape[0] / wiki_df["in edges"].shape[0]) * 100), "%")

29.691253735784134 %
29.83103724074137 %


In addition, one can see that part of the skewness of the degree distribution (see bottom groups) is due to pages that have either no incoming or outgoing edges. Having no incoming edges can be explained by pages that are the "entry-page" to this network of pages (i.e. referred to by somebody googling it / typing it in their address bar) while having no outgoing edges can be explained by pages that simply have no links on them (i.e. pages showing files etc.). These possible explanations are, however, just assumptions / guesses. There is no metadata about the pages available, so we can actually not confirm these hypotheses by more in-depth evaluations.

### <font color="darkgreen">2. Static Network Analysis:</font> Diameter